In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install langchain-openai
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.9 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

import os
import requests
import langchain
import yfinance as yf
from langchain_groq import ChatGroq
import ast
import datetime
import pandas as pd
import time

In [6]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [7]:
#initialize OpenAI LLM

openai_llm = ChatOpenAI(

    model_name="gpt-4",  # or "gpt-3.5-turbo" for a less expensive option

    temperature=0,

    api_key=userdata.get('OPENAI_API_KEY'))

In [8]:
#initialize Groq LLM

# For LLaMA 3 8B model

groq_llm_llama3_8b = ChatGroq(

    model_name="llama3-8b-8192",

    temperature=0,

    api_key=userdata.get('GROQ_API_KEY'))


In [9]:
# For LLaMA 3 70B model

groq_llm_llama3_70b = ChatGroq(

    model_name="llama3-70b-8192",

    temperature=0,

    api_key=userdata.get('GROQ_API_KEY'))

In [10]:
# Function to select LLM based on user preference

def get_llm(provider="openai", model="default"):

    if provider.lower() == "openai":

        return openai_llm

    elif provider.lower() == "groq":

        if model.lower() == "llama3-70b":

            return groq_llm_llama3_70b

        else:

            return groq_llm_llama3_8b

    else:

        # Default to OpenAI if provider not recognized

        return openai_llm


In [11]:
# Task 1: Implement Stock Price Lookup Tool
def get_stock_price(symbol):
    """
    Fetches the latest stock price for the given symbol.

    Args:
        symbol (str): Stock ticker symbol (e.g., 'AAPL')

    Returns:
        dict: Stock information including current price and daily change
    """
    try:
        # Clean the symbol to remove any unwanted characters
        symbol = symbol.strip().upper()

        # Use yfinance to fetch current stock data
        stock = yf.Ticker(symbol)
        stock_info = stock.info

        # Get historical data to calculate daily change
        hist = stock.history(period="2d")

        if len(hist) < 2:  #if less historical data
            current_price = stock_info.get('currentPrice', stock_info.get('regularMarketPrice', 0)) #get current price from stock_info dict
            previous_close = stock_info.get('previousClose', current_price)
        else:
            current_price = hist['Close'].iloc[-1]  #get current price from hist df
            previous_close = hist['Close'].iloc[-2]

        # Calculate daily change and percentage
        daily_change = current_price - previous_close
        daily_change_percent = (daily_change / previous_close) * 100 if previous_close > 0 else 0

        # Return formatted result
        result = {
            "symbol": symbol,
            "current_price": round(current_price, 2),
            "daily_change": round(daily_change, 2),
            "daily_change_percent": round(daily_change_percent, 2),
            "previous_close": round(previous_close, 2),
            "currency": stock_info.get('currency', 'USD'),
            "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        return result

    except Exception as e:
        return f"Error retrieving stock price for {symbol}: {str(e)}"

In [12]:
get_stock_price('AAPL')

{'symbol': 'AAPL',
 'current_price': np.float64(198.15),
 'daily_change': np.float64(7.73),
 'daily_change_percent': np.float64(4.06),
 'previous_close': np.float64(190.42),
 'currency': 'USD',
 'timestamp': '2025-04-12 21:40:38'}

In [13]:
# Create the stock price lookup tool

stock_price_tool = Tool(

    name="StockPriceLookup",

    func=get_stock_price,

    description="Useful for getting the current price of a stock and relevant info. Input should be a valid stock ticker symbol."

)


In [14]:
# Task 2: Implement Portfolio Rebalancing Tool
def rebalance_portfolio(portfolio_str):

    """
    Takes a portfolio string representation and suggests rebalancing actions.

    Args:
        portfolio_str (str): String representation of portfolio, e.g., "{'AAPL': 0.5, 'TSLA': 0.3, 'GOOGL': 0.2}"

    Returns:
        str: Rebalancing recommendations for equal weights strategy
    """

    try:

        # Parse the portfolio string into a dictionary
        if isinstance(portfolio_str, str):
            portfolio = ast.literal_eval(portfolio_str)
        else:
            portfolio = portfolio_str

        # Validate portfolio structure
        if not isinstance(portfolio, dict):
            return "Error: Portfolio must be provided as a dictionary mapping stock symbols to weights."

        # Check if weights sum to approximately 1 (allowing for small floating point errors)
        total_weight = sum(portfolio.values())
        if not 0.99 <= total_weight <= 1.01:
            return f"Warning: Portfolio weights sum to {total_weight}, not 1.0. This analysis assumes the provided weights represent the full portfolio."

        # Get current stock prices
        stock_prices = {}
        for symbol in portfolio:
            price_data = get_stock_price(symbol)
            if isinstance(price_data, dict) and 'current_price' in price_data:
                stock_prices[symbol] = price_data['current_price']
            else:
                return f"Failed to get price for {symbol}. Error: {price_data}"

        n = len(portfolio)

        if n == 0:
            return "Portfolio is empty."

        # Calculate target equal weight
        target_weight = 1 / n

        # Build recommendation list
        actions = []

        for symbol, weight in portfolio.items():
            diff = round(weight - target_weight, 4)

            if diff > 0.01:
                #action = f"🔻 Sell {abs(diff) * 100:.2f}% of {symbol}"
                action = f"SELL {symbol} at ${stock_prices[symbol]}: DECREASE allocation by {diff:.2%}"

            elif diff < 0.01:
                #action = f"🔼 Buy {abs(diff) * 100:.2f}% more of {symbol}"
                action = f"BUY {symbol} at ${stock_prices[symbol]}: INCREASE allocation by {diff:.2%}"
            else:
               # action = f"✅ {symbol} is already balanced"
                action = f"HOLD {symbol} at ${stock_prices[symbol]}: Current allocation {portfolio[symbol]}"

            actions.append(action)



        # Generate recommendations summary
        summary = f"Portfolio Rebalancing Analysis (Equal Weight Strategy)\n"
        summary += f"Target allocation per stock: {target_weight:.2%}\n\n"
        summary += "Current Allocations:\n"
        for symbol in portfolio:
            summary += f"- {symbol}: {portfolio[symbol]:.2%}\n"

        summary += "\nRecommended Actions:\n"
        for action in actions:
            summary += f"- {action}\n"

        summary += "\nAfter rebalancing, each stock will represent approximately {:.2%} of your portfolio.".format(target_weight)

        return summary

    except Exception as e:
        return f"Error analyzing portfolio: {str(e)}"

In [15]:
rec = rebalance_portfolio("{'AAPL': 0.5, 'TSLA': 0.3, 'GOOGL': 0.2}")
print(rec)

Portfolio Rebalancing Analysis (Equal Weight Strategy)
Target allocation per stock: 33.33%

Current Allocations:
- AAPL: 50.00%
- TSLA: 30.00%
- GOOGL: 20.00%

Recommended Actions:
- SELL AAPL at $198.15: DECREASE allocation by 16.67%
- BUY TSLA at $252.31: INCREASE allocation by -3.33%
- BUY GOOGL at $157.14: INCREASE allocation by -13.33%

After rebalancing, each stock will represent approximately 33.33% of your portfolio.


In [16]:
# Create the portfolio rebalancing tool

rebalance_tool = Tool(

    name="PortfolioRebalancer",

    func=rebalance_portfolio,

    description="Analyzes a portfolio and suggests rebalancing actions. Input should be a dictionary mapping stock symbols to their current weight in the portfolio."

)

In [17]:
# Task 3: Implement Market Trend Analysis Tool
def market_trend_analysis(_input: str = "") -> str:
    """
    Fetches stock market index trends over the past week.

    Returns:
        str: Analysis of market trends
    """
    try:
        # Define major indices to analyze
        indices = {
            "S&P 500": "^GSPC",
            "Dow Jones": "^DJI",
            "NASDAQ": "^IXIC"
        }

        results = {}

        # Get data for each index
        for index_name, ticker in indices.items():
            # Fetch data for the past 7 days

            data = yf.download(ticker, period="7d")

            # Flatten MultiIndex columns if needed
            if isinstance(data.columns, pd.MultiIndex):
              data.columns = data.columns.get_level_values(0)



            if len(data) < 2:
                results[index_name] = f"Insufficient data for {index_name}"
                continue

            # Calculate start and end price of index
            start_price = data['Close'].iloc[0]
            end_price = data['Close'].iloc[-1]



            # Calculate 7-day return
            price_change = end_price - start_price
            percent_change = (price_change / start_price) * 100


            # Calculate volatility (standard deviation of returns)
            daily_returns = data['Close'].pct_change().dropna()
            volatility = daily_returns.std() * 100

            # 10-day moving average (if available)
            ma_data = yf.download(ticker, period="20d")

            if isinstance(ma_data.columns, pd.MultiIndex):
              ma_data.columns = ma_data.columns.get_level_values(0)

            if len(ma_data) >= 10:
                ma10 = ma_data['Close'].rolling(window=10).mean().iloc[-1]
                ma_comparison = "above" if end_price > ma10 else "below"
            else:
                ma_comparison = "Can't calculate 10-day moving average from data"
                ma10 = None

            # Store results

            results[index_name] = {
                "ticker": ticker,
                "start_price": round(start_price, 2),
                "end_price": round(end_price, 2),
                "price_change": round(price_change, 2),
                "percent_change": round(percent_change, 2),
                "volatility": round(volatility, 2),
                "ma10": round(ma10, 2) if ma10 is not None else None,
                "ma_comparison": ma_comparison
            }

        # Generate analysis text
        analysis = "\n\n Market Trend Analysis (Past Week)\n\n"



        # Overall market sentiment
        positive_indices = sum(1 for index in results if isinstance(results[index], dict) and results[index]['percent_change'] > 0)
        total_indices = sum(1 for index in results if isinstance(results[index], dict))


        if positive_indices > total_indices / 2:
            overall_sentiment = "Overall market sentiment appears positive with most major indices gaining value."
        elif positive_indices < total_indices / 2:
            overall_sentiment = "Overall market sentiment appears negative with most major indices losing value."
        else:
            overall_sentiment = "Market sentiment is mixed with an equal number of indices gaining and losing value."

        analysis += overall_sentiment + "\n\n"



        # Individual index analysis
        for index_name, data in results.items():
            if isinstance(data, str):
                analysis += f"{index_name}: {data}\n\n"
                continue

            analysis += f"{index_name} ({data['ticker']}):\n"


            # Price movement
            if data['percent_change'] > 0:
                movement = f"+{data['percent_change']}% over the past week (START PRICE: ${data['start_price']} END PRICE: ${data['end_price']})"
            elif data['percent_change'] < 0:
                movement = f"-{abs(data['percent_change'])}% over the past week (START PRICE: ${data['start_price']} END PRICE: ${data['end_price']})"
            else:
                movement = f"No change over the past week (START PRICE: ${data['start_price']} END PRICE: ${data['end_price']})"

            analysis += f"- 7-DAY Returns: {movement}\n"



            # Volatility
            if data['volatility'] < 1:
                vol_assessment = "very low volatility"
            elif data['volatility'] < 1.5:
                vol_assessment = "low volatility"
            elif data['volatility'] < 2.5:
                vol_assessment = "moderate volatility"
            else:
                vol_assessment = "high volatility"

            analysis += f"- Volatility: {data['volatility']}% daily (indicating {vol_assessment})\n"



            #MA comparison
            if data['ma10'] is not None:
                analysis += f"- 10-day Moving Average: Current Price is {data['ma_comparison']} the 10-day moving average of ${data['ma10']}\n"

            analysis += "\n"


        # Add timestamp
        analysis += f"Analysis generated on {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

        return analysis

    except Exception as e:
        return f"Error analyzing market trends: {str(e)}"

In [18]:
print(market_trend_analysis())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

YF.download() has changed argument auto_adjust default to True



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



 Market Trend Analysis (Past Week)

Overall market sentiment appears negative with most major indices losing value.

S&P 500 (^GSPC):
- 7-DAY Returns: -0.61% over the past week (START PRICE: $5396.52 END PRICE: $5363.36)
- Volatility: 5.37% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $5351.98

Dow Jones (^DJI):
- 7-DAY Returns: -0.82% over the past week (START PRICE: $40545.93 END PRICE: $40212.71)
- Volatility: 4.53% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $40110.38

NASDAQ (^IXIC):
- 7-DAY Returns: +1.05% over the past week (START PRICE: $16550.61 END PRICE: $16724.46)
- Volatility: 6.46% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $16559.65

Analysis generated on 2025-04-12 21:41:04


In [19]:
# Create the market trend analysis tool

trend_tool = Tool(

    name="MarketTrendAnalyzer",

    func=market_trend_analysis,

    description="Provides an analysis of recent market trends. No input required."

)


In [81]:
# Function to create and run an agent with the selected LLM

def create_and_run_agent(llm_provider="openai", llm_model="default", query=""):

    llm = get_llm(llm_provider, llm_model)

    tools = [stock_price_tool, rebalance_tool, trend_tool]

    agent = initialize_agent(

        tools=tools,

        llm=llm,

        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,

        verbose=True

    )

    # Measure response time
    start_time = time.time()
    result = agent.run(query)
    end_time = time.time()

    response_time = end_time - start_time

    return {
        "result": result,
        "response_time": response_time
    }


In [80]:
# Test cases with different LLMs

def run_test_cases():

    user_portfolio_1 = {"AAPL": 0.50, "TSLA": 0.30, "GOOGL": 0.20}

    user_portfolio_2 = {"MSFT": 0.25, "NVDA": 0.25, "AMZN": 0.25, "META": 0.25}

    performance_metrics = {
        "openai": {"portfolio_1": {}, "portfolio_2": {}},
        "groq_llama3_8b": {"portfolio_1": {}, "portfolio_2": {}},
        "groq_llama3_70b": {"portfolio_1": {}, "portfolio_2": {}}
    }



    # Test with OpenAI

    print("OpenAI GPT-4 Results:")

    response_1 = create_and_run_agent("openai", "default", f"Analyze this portfolio and recommend changes: {user_portfolio_1}")

    print("Portfolio 1:" )
    print(f"Response time: {response_1['response_time']:.2f} seconds")
    print("Result:", response_1["result"])

    response_2 = create_and_run_agent("openai", "default", f"Analyze this portfolio and recommend changes: {user_portfolio_2}")

    print("Portfolio 2:" )
    print(f"Response time: {response_2['response_time']:.2f} seconds")
    print("Result:", response_2["result"])

    print(f"Average Response Time:  {(response_1['response_time']+response_2['response_time'])/2:.2f} seconds")


    #print("Portfolio 1:", create_and_run_agent("openai", "default", f"Analyze this portfolio and recommend changes: {user_portfolio_1}"))

    #print("Portfolio 2:", create_and_run_agent("openai", "default", f"Analyze this portfolio and recommend changes: {user_portfolio_2}"))

    performance_metrics["openai"]["portfolio_1"] = {
        "response_time": response_1["response_time"],
        "result": response_1["result"]
    }

    performance_metrics["openai"]["portfolio_2"] = {
        "response_time": response_2["response_time"],
        "result": response_2["result"]
    }

    print('-' * 25)
    print("\n \n")



    # Test with Groq LLaMA3-8B

    print("\nGroq LLaMA3-8B Results:")

    response_1 = create_and_run_agent("groq", "llama3-8b", f"Analyze this portfolio and recommend changes: {user_portfolio_1}")

    print("Portfolio 1:" )
    print(f"Response time: {response_1['response_time']:.2f} seconds")
    print("Result:", response_1["result"])

    response_2 = create_and_run_agent("groq", "llama3-8b", f"Analyze this portfolio and recommend changes: {user_portfolio_2}")

    print("Portfolio 2:" )
    print(f"Response time: {response_2['response_time']:.2f} seconds")
    print("Result:", response_2["result"])

    print(f"Average Response Time:  {(response_1['response_time']+response_2['response_time'])/2:.2f} seconds")

    performance_metrics["groq_llama3_8b"]["portfolio_1"] = {
        "response_time": response_1["response_time"],
        "result": response_1["result"]
    }

    performance_metrics["groq_llama3_8b"]["portfolio_2"] = {
        "response_time": response_2["response_time"],
        "result": response_2["result"]
    }

    print('-' * 25)
    print("\n \n")


    # Test with Groq LLaMA3-70B

    print("\nGroq LLaMA3-70B Results:")

    response_1 = create_and_run_agent("groq", "llama3-70b", f"Analyze this portfolio and recommend changes: {user_portfolio_1}")

    print("Portfolio 1:" )
    print(f"Response time: {response_1['response_time']:.2f} seconds")
    print("Result:", response_1["result"])

    response_2 = create_and_run_agent("groq", "llama3-70b", f"Analyze this portfolio and recommend changes: {user_portfolio_2}")

    print("Portfolio 2:" )
    print(f"Response time: {response_2['response_time']:.2f} seconds")
    print("Result:", response_2["result"])

    print(f"Average Response Time:  {(response_1['response_time']+response_2['response_time'])/2:.2f} seconds")

    performance_metrics["groq_llama3_70b"]["portfolio_1"] = {
        "response_time": response_1["response_time"],
        "result": response_1["result"]
    }

    performance_metrics["groq_llama3_70b"]["portfolio_2"] = {
        "response_time": response_2["response_time"],
        "result": response_2["result"]
    }

    print('-' * 25)
    print("\n \n")

    return performance_metrics

In [72]:
def generate_performance_report(metrics,fname):
    """Generate and save a performance comparison report"""

    report = "# LLM Performance Comparison for Financial Portfolio Analysis\n\n"

    # Create comparison table
    report += "## Response Time Comparison (seconds)\n\n"
    report += "| Model | Portfolio 1 | Portfolio 2 | Average |\n"
    report += "|-------|------------|------------|--------|\n"

    for model in metrics:
        p1_time = metrics[model]["portfolio_1"]["response_time"]
        p2_time = metrics[model]["portfolio_2"]["response_time"]
        avg_time = (p1_time + p2_time) / 2

        model_name = "OpenAI GPT-4" if model == "openai" else model
        report += f"| {model_name} | {p1_time:.2f} | {p2_time:.2f} | {avg_time:.2f} |\n"

    fpath = '/content/gdrive/MyDrive/Colab Notebooks/'+fname+'.md'

    #fpath = '/content/gdrive/MyDrive/Colab Notebooks/llm_performance_comparison1.md'

    # Save report to file
    with open(fpath, "w") as f:
        f.write(report)
    print(report)
    print("\nPerformance report generated and saved to 'llm_performance_comparison.md'")


In [74]:

if __name__ == "__main__":
    pmetrics = run_test_cases()

OpenAI GPT-4 Results:


> Entering new AgentExecutor chain...
I need to analyze the portfolio and suggest rebalancing actions.
Action: PortfolioRebalancer
Action Input: {'AAPL': 0.5, 'TSLA': 0.3, 'GOOGL': 0.2}
Observation: Portfolio Rebalancing Analysis (Equal Weight Strategy)
Target allocation per stock: 33.33%

Current Allocations:
- AAPL: 50.00%
- TSLA: 30.00%
- GOOGL: 20.00%

Recommended Actions:
- SELL AAPL at $198.15: DECREASE allocation by 16.67%
- BUY TSLA at $252.31: INCREASE allocation by -3.33%
- BUY GOOGL at $157.14: INCREASE allocation by -13.33%

After rebalancing, each stock will represent approximately 33.33% of your portfolio.
Thought:I now know the final answer
Final Answer: The recommended changes to your portfolio are to sell some AAPL stocks to decrease its allocation by 16.67%, buy more TSLA stocks to increase its allocation by 3.33%, and buy more GOOGL stocks to increase its allocation by 13.33%. After these changes, each stock will represent approximately 33.33%

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Thought: I was expecting some changes to be recommended, but it seems that the portfolio is already perfectly balanced. I'll think about what else I can do.

Action: MarketTrendAnalyzer

Action Input: None



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Observation: 

 Market Trend Analysis (Past Week)

Overall market sentiment appears negative with most major indices losing value.

S&P 500 (^GSPC):
- 7-DAY Returns: -0.61% over the past week (START PRICE: $5396.52 END PRICE: $5363.36)
- Volatility: 5.37% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $5351.98

Dow Jones (^DJI):
- 7-DAY Returns: -0.82% over the past week (START PRICE: $40545.93 END PRICE: $40212.71)
- Volatility: 4.53% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $40110.38

NASDAQ (^IXIC):
- 7-DAY Returns: +1.05% over the past week (START PRICE: $16550.61 END PRICE: $16724.46)
- Volatility: 6.46% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $16559.65

Analysis generated on 2025-04-12 23:16:11
Thought:Thought: It seems that the overall market sentiment is negative, with mos

...


Observation: 

 Market Trend Analysis (Past Week)

Overall market sentiment appears negative with most major indices losing value.

S&P 500 (^GSPC):
- 7-DAY Returns: -0.61% over the past week (START PRICE: $5396.52 END PRICE: $5363.36)
- Volatility: 5.37% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $5351.98

Dow Jones (^DJI):
- 7-DAY Returns: -0.82% over the past week (START PRICE: $40545.93 END PRICE: $40212.71)
- Volatility: 4.53% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $40110.38

NASDAQ (^IXIC):
- 7-DAY Returns: +1.05% over the past week (START PRICE: $16550.61 END PRICE: $16724.46)
- Volatility: 6.46% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $16559.65

Analysis generated on 2025-04-12 23:16:55
Thought:Thought: I now know the final answer.

Final Answer: The portfolio is alr

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Thought: It seems like the portfolio is already well-balanced, with each stock having a 25% allocation. However, I'd like to get a better understanding of the current market trends to see if there are any opportunities to optimize the portfolio further.

Action: MarketTrendAnalyzer
Action Input: 


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Observation: 

 Market Trend Analysis (Past Week)

Overall market sentiment appears negative with most major indices losing value.

S&P 500 (^GSPC):
- 7-DAY Returns: -0.61% over the past week (START PRICE: $5396.52 END PRICE: $5363.36)
- Volatility: 5.37% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $5351.98

Dow Jones (^DJI):
- 7-DAY Returns: -0.82% over the past week (START PRICE: $40545.93 END PRICE: $40212.71)
- Volatility: 4.53% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $40110.38

NASDAQ (^IXIC):
- 7-DAY Returns: +1.05% over the past week (START PRICE: $16550.61 END PRICE: $16724.46)
- Volatility: 6.46% daily (indicating high volatility)
- 10-day Moving Average: Current Price is above the 10-day moving average of $16559.65

Analysis generated on 2025-04-12 23:17:24
Thought:Thought: Now that I have an understanding of the current market trends, I

In [60]:
print(pmetrics)


{'openai': {'portfolio_1': {}, 'portfolio_2': {}}, 'groq_llama3_8b': {'portfolio_1': {'response_time': 33.882779598236084, 'result': 'The recommended actions to rebalance the portfolio are to sell AAPL at $198.15 to decrease its allocation by 16.67%, buy TSLA at $252.31 to increase its allocation by -3.33%, and buy GOOGL at $157.14 to increase its allocation by -13.33%.'}, 'portfolio_2': {'response_time': 85.66860485076904, 'result': "The portfolio is already perfectly balanced, with each stock having an equal weight of 25%. The overall market sentiment is negative, with most major indices losing value over the past week. Microsoft's stock price has increased by 1.86% over the past day, which is a relatively positive trend."}}, 'groq_llama3_70b': {'portfolio_1': {'response_time': 5.304510593414307, 'result': 'Based on the analysis, I recommend rebalancing the portfolio by selling AAPL to decrease its allocation by 16.67%, buying TSLA to increase its allocation by 3.33%, and buying GOOG

In [82]:
!python --version

Python 3.11.12


In [75]:
filename = 'llm_performance_comparison2'
generate_performance_report(pmetrics,filename)

# LLM Performance Comparison for Financial Portfolio Analysis

## Response Time Comparison (seconds)

| Model | Portfolio 1 | Portfolio 2 | Average |
|-------|------------|------------|--------|
| OpenAI GPT-4 | 6.61 | 5.09 | 5.85 |
| groq_llama3_8b | 2.57 | 68.05 | 35.31 |
| groq_llama3_70b | 5.02 | 65.03 | 35.02 |


Performance report generated and saved to 'llm_performance_comparison.md'


In [78]:
def print_results(perf_metrics):
  for model, portfolios in perf_metrics.items():
        model_name = "OpenAI GPT-4" if model == "openai" else model
        print(f"\nModel: {model_name}")
        for pname, data in portfolios.items():
            print(f"\n{pname.replace('_', ' ').title()}")
            print(f"Response Time: {round(data['response_time'], 2)} seconds")
            print(f"Result:\n{data['result']}")

In [79]:
print_results(pmetrics)


Model: OpenAI GPT-4

Portfolio 1
Response Time: 6.61 seconds
Result:
The recommended changes to your portfolio are to sell some AAPL stocks to decrease its allocation by 16.67%, buy more TSLA stocks to increase its allocation by 3.33%, and buy more GOOGL stocks to increase its allocation by 13.33%. After these changes, each stock will represent approximately 33.33% of your portfolio.

Portfolio 2
Response Time: 5.09 seconds
Result:
No changes are required in your portfolio. It is already balanced with each stock representing approximately 25.00% of your portfolio.

Model: groq_llama3_8b

Portfolio 1
Response Time: 2.57 seconds
Result:
The recommended actions to rebalance the portfolio are to sell AAPL at $198.15 to decrease its allocation by 16.67%, buy TSLA at $252.31 to increase its allocation by -3.33%, and buy GOOGL at $157.14 to increase its allocation by -13.33%.

Portfolio 2
Response Time: 68.05 seconds
Result:
The portfolio is already perfectly balanced, with each stock having